# AI Lab Assignment -2 
# Topic: Apply Hill Climbing Heuristic Search on TSP

---



> Name- Rugved Chavan 

> Reg-no- 199303045

> cce sec-A




In [15]:
print("Hello World!")
!python --version

Hello World!
Python 3.7.12


## Hill Climbing

---


Hill-climbing is a local search method that starts with a solution and tries to improve it until no more improvement can be made.
Hill-climbing is a simple algorithm for quickly finding a good solution without requiring a large amount of memory.

> Types of Hill-Climbing Techniques:
1. stochastic hill climbing, 
2. first-choice hill climbing, 
3. Steepest hill climbing,
4. random-restart hill climbing and more custom variants. 

> Problem of Hill-Climbing Techniques:
1. Local maxima (Solution: Backtracking.)
2. Plateu (Solution: A big jump in any direction.)
3. Ridges (Solution: Trying multiple paths at once or descending to the sharpest point and then climbing back up.)

> Disadvantages of Hill Climbing
- It is not an efficient method.
- It is not suited to problems where the value of the heuristic function drops off suddenly
when the solution may be in sight.
- It is a local method as it looks at the immediate solution and decides about the next
step to be taken rather than exploring all consequences before taking a move.

# Traveling Salesman Problem (TSP)

The problem was first proposed in 1930, and it is now one of the most studied in the field of optimization. Many optimization approaches use it as a benchmark.

---

A salesman and a group of cities are the heart of the travelling salesman problem. The salesperson must travel to each of the cities, beginning in one (for example, his hometown) and ending in the same. The problem's difficulty is that the travelling salesman must reduce the entire length of the journey.




# Code & Procedure in brief

---



Import libraries:

In [3]:
import random
import copy

This class represent a state:

In [4]:
class State:
    # Create a new state
    def __init__(self, route:[], distance:int=0):
        self.route = route
        self.distance = distance
    # Compare states
    def __eq__(self, other):
        for i in range(len(self.route)):
            if(self.route[i] != other.route[i]):
                return False
        return True
    # Sort states
    def __lt__(self, other):
         return self.distance < other.distance
    # Print a state
    def __repr__(self):
        return ('({0},{1})\n'.format(self.route, self.distance))
    # Create a shallow copy
    def copy(self):
        return State(self.route, self.distance)
    # Create a deep copy
    def deepcopy(self):
        return State(copy.deepcopy(self.route), copy.deepcopy(self.distance))
    # Update distance
    def update_distance(self, matrix, home):
        
        # Reset distance
        self.distance = 0
        # Keep track of departing city
        from_index = home
        # Loop all cities in the current route
        for i in range(len(self.route)):
            self.distance += matrix[from_index][self.route[i]]
            from_index = self.route[i]
        # Add the distance back to home
        self.distance += matrix[from_index][home]


This class represent a city (used when we need to delete cities)

In [5]:

class City:
    # Create a new city
    def __init__(self, index:int, distance:int):
        self.index = index
        self.distance = distance
    # Sort cities
    def __lt__(self, other):
         return self.distance < other.distance

Get the best random solution from a population

In [6]:
def get_random_solution(matrix:[], home:int, city_indexes:[], size:int, use_weights=False):
    # Create a list with city indexes
    cities = city_indexes.copy()
    # Remove the home city
    cities.pop(home)
    # Create a population
    population = []
    for i in range(size):
        if(use_weights == True):
            state = get_random_solution_with_weights(matrix, home)
        else:
            # Shuffle cities at random
            random.shuffle(cities)
            # Create a state
            state = State(cities[:])
            state.update_distance(matrix, home)
        # Add an individual to the population
        population.append(state)
    # Sort population
    population.sort()
    # Return the best solution
    return population[0]

Get best solution by distance

In [7]:
def get_best_solution_by_distance(matrix:[], home:int):
    
    # Variables
    route = []
    from_index = home
    length = len(matrix) - 1
    # Loop until route is complete
    while len(route) < length:
         # Get a matrix row
        row = matrix[from_index]
        # Create a list with cities
        cities = {}
        for i in range(len(row)):
            cities[i] = City(i, row[i])
        # Remove cities that already is assigned to the route
        del cities[home]
        for i in route:
            del cities[i]
        # Sort cities
        sorted = list(cities.values())
        sorted.sort()
        # Add the city with the shortest distance
        from_index = sorted[0].index
        route.append(from_index)
    # Create a new state and update the distance
    state = State(route)
    state.update_distance(matrix, home)
    # Return a state
    return state

Get a random solution by using weights

In [8]:

def get_random_solution_with_weights(matrix:[], home:int):
    
    # Variables
    route = []
    from_index = home
    length = len(matrix) - 1
    # Loop until route is complete
    while len(route) < length:
         # Get a matrix row
        row = matrix[from_index]
        # Create a list with cities
        cities = {}
        for i in range(len(row)):
            cities[i] = City(i, row[i])
        # Remove cities that already is assigned to the route
        del cities[home]
        for i in route:
            del cities[i]
        # Get the total weight
        total_weight = 0
        for key, city in cities.items():
            total_weight += city.distance
        # Add weights
        weights = []
        for key, city in cities.items():
            weights.append(total_weight / city.distance)
        # Add a city at random
        from_index = random.choices(list(cities.keys()), weights=weights)[0]
        route.append(from_index)
    # Create a new state and update the distance
    state = State(route)
    state.update_distance(matrix, home)
    # Return a state
    return state

Mutate a solution

In [9]:
def mutate(matrix:[], home:int, state:State, mutation_rate:float=0.01):
    
    # Create a copy of the state
    mutated_state = state.deepcopy()
    # Loop all the states in a route
    for i in range(len(mutated_state.route)):
        # Check if we should do a mutation
        if(random.random() < mutation_rate):
            # Swap two cities
            j = int(random.random() * len(state.route))
            city_1 = mutated_state.route[i]
            city_2 = mutated_state.route[j]
            mutated_state.route[i] = city_2
            mutated_state.route[j] = city_1
    # Update the distance
    mutated_state.update_distance(matrix, home)
    # Return a mutated state
    return mutated_state

Hill climbing algorithm

In [10]:
def hill_climbing(matrix:[], home:int, initial_state:State, max_iterations:int, mutation_rate:float=0.01):
    # Keep track of the best state
    best_state = initial_state
    # An iterator can be used to give the algorithm more time to find a solution
    iterator = 0
    # Create an infinite loop
    while True:
        # Mutate the best state
        neighbor = mutate(matrix, home, best_state, mutation_rate)
        # Check if the distance is less than in the best state
        if(neighbor.distance >= best_state.distance):
            iterator += 1
            if (iterator > max_iterations):
                break
        if(neighbor.distance < best_state.distance):
            best_state = neighbor
    # Return the best state
    return best_state

The main entry point for this module

In [13]:
def main():
    # Cities to travel
    cities = ['New York', 'Los Angeles', 'Chicago', 'Minneapolis', 'Denver', 'Dallas', 'Seattle', 'Boston', 'San Francisco', 'St. Louis', 'Houston', 'Phoenix', 'Salt Lake City']
    city_indexes = [0,1,2,3,4,5,6,7,8,9,10,11,12]
    # Index of start location
    home = 2 # Chicago
    # Max iterations
    max_iterations = 1000
    # Distances in miles between cities, same indexes (i, j) as in the cities array
    matrix = [[0, 2451, 713, 1018, 1631, 1374, 2408, 213, 2571, 875, 1420, 2145, 1972],
            [2451, 0, 1745, 1524, 831, 1240, 959, 2596, 403, 1589, 1374, 357, 579],
            [713, 1745, 0, 355, 920, 803, 1737, 851, 1858, 262, 940, 1453, 1260],
            [1018, 1524, 355, 0, 700, 862, 1395, 1123, 1584, 466, 1056, 1280, 987],
            [1631, 831, 920, 700, 0, 663, 1021, 1769, 949, 796, 879, 586, 371],
            [1374, 1240, 803, 862, 663, 0, 1681, 1551, 1765, 547, 225, 887, 999],
            [2408, 959, 1737, 1395, 1021, 1681, 0, 2493, 678, 1724, 1891, 1114, 701],
            [213, 2596, 851, 1123, 1769, 1551, 2493, 0, 2699, 1038, 1605, 2300, 2099],
            [2571, 403, 1858, 1584, 949, 1765, 678, 2699, 0, 1744, 1645, 653, 600],
            [875, 1589, 262, 466, 796, 547, 1724, 1038, 1744, 0, 679, 1272, 1162],
            [1420, 1374, 940, 1056, 879, 225, 1891, 1605, 1645, 679, 0, 1017, 1200],
            [2145, 357, 1453, 1280, 586, 887, 1114, 2300, 653, 1272, 1017, 0, 504],
            [1972, 579, 1260, 987, 371, 999, 701, 2099, 600, 1162, 1200, 504, 0]]
    # Get the best route by distance
    state = get_best_solution_by_distance(matrix, home)
    print('-- Best solution by distance --')
    print(cities[home], end='')
    for i in range(0, len(state.route)):
       print(' -> ' + cities[state.route[i]], end='')
    print(' -> ' + cities[home], end='')
    print('\n\nTotal distance: {0} miles'.format(state.distance))
    print()
    # Get the best random route
    state = get_random_solution(matrix, home, city_indexes, 100)
    print('-- Best random solution --')
    print(cities[home], end='')
    for i in range(0, len(state.route)):
       print(' -> ' + cities[state.route[i]], end='')
    print(' -> ' + cities[home], end='')
    print('\n\nTotal distance: {0} miles'.format(state.distance))
    print()
    # Get a random solution with weights
    state = get_random_solution(matrix, home, city_indexes, 100, use_weights=True)
    print('-- Best random solution with weights --')
    print(cities[home], end='')
    for i in range(0, len(state.route)):
       print(' -> ' + cities[state.route[i]], end='')
    print(' -> ' + cities[home], end='')
    print('\n\nTotal distance: {0} miles'.format(state.distance))
    print()
    # Run hill climbing to find a better solution
    state = get_best_solution_by_distance(matrix, home)
    state = hill_climbing(matrix, home, state, 1000, 0.1)
    print('-- Hill climbing solution --')
    print(cities[home], end='')
    for i in range(0, len(state.route)):
       print(' -> ' + cities[state.route[i]], end='')
    print(' -> ' + cities[home], end='')
    print('\n\nTotal distance: {0} miles'.format(state.distance))
    print("Auther: Rugved Chavan, CCE-A, 199303045")
    print()


Main driver run
# Results:

In [14]:
if __name__ == "__main__": main()

-- Best solution by distance --
Chicago -> St. Louis -> Minneapolis -> Denver -> Salt Lake City -> Phoenix -> Los Angeles -> San Francisco -> Seattle -> Dallas -> Houston -> New York -> Boston -> Chicago

Total distance: 8131 miles

-- Best random solution --
Chicago -> St. Louis -> Seattle -> San Francisco -> Los Angeles -> Phoenix -> Houston -> Salt Lake City -> Denver -> Minneapolis -> Boston -> New York -> Dallas -> Chicago

Total distance: 10225 miles

-- Best random solution with weights --
Chicago -> Dallas -> Houston -> St. Louis -> Denver -> Salt Lake City -> Phoenix -> Los Angeles -> San Francisco -> Seattle -> Boston -> New York -> Minneapolis -> Chicago

Total distance: 8895 miles

-- Hill climbing solution --
Chicago -> St. Louis -> Minneapolis -> Denver -> Salt Lake City -> Seattle -> Los Angeles -> San Francisco -> Phoenix -> Dallas -> Houston -> New York -> Boston -> Chicago

Total distance: 8111 miles
Auther: Rugved Chavan, CCE-A, 199303045



# Remark:
As an initial solution, I choose to utilise the best solution by distance; the best solution is mutated in each iteration, and a mutated solution will be the new best solution if the overall distance is less than the current best solution's distance. Extra iterations are being used to give the algorithm more time to come up with a better solution. 7293 miles is the best solution.

# Reference:

---



[1] http://www.simplynotes.in/e-notes/mcabca/artificical-intelligence/hill-climbing/#:~:text=Disadvantages%20of%20Hill%20Climbing&text=It%20is%20not%20suited%20to,consequences%20before%20taking%20a%20move.

[2] https://www.annytab.com/hill-climbing-search-algorithm-in-python/

[3]https://www.javatpoint.com/travelling-sales-person-problem

  **Thankyou **
  
  -Rugved Chavan

In [16]:
!git init

Initialized empty Git repository in /content/.git/


In [17]:
!git add .

In [18]:
! git commit -m "done"


*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@4f258ce7b770.(none)')


In [20]:
!git config --global user.email "rugvedm12@fmail.com"